## Topic Modeling and Latent Dirichlet Allocation (LDA) in Python

Topic modeling is a type of statistical modeling for discovering the abstract “topics” that occur in a collection of documents. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.
Here we are going to apply LDA to a set of documents and split them into topics. Let’s get started!
#### The Data
The data set we’ll use is a list of over one million news headlines published over a period of 15 years and can be downloaded from Kaggle.

In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


## Data Pre-processing
We will perform the following steps:
* Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
Words that have fewer than 3 characters are removed.
* All stopwords are removed.
* Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are stemmed — words are reduced to their root form.

### Loading gensim and nltk libraries

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

D:\anaconda\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
D:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajumo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [9]:
stemmer=SnowballStemmer(language='english')

**Write a function to perform lemmatize and stem preprocessing steps on the data set.**

In [10]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

**Select a document to preview after preprocessing.** 

In [11]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


### Preprocess the headline text, saving the results as ‘processed_docs’

In [14]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [16]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


### Gensim filter_extremes

* Filter out tokens that appear in
* less than 15 documents (absolute number) or
* more than 0.5 documents (fraction of total corpus size, not absolute number).
* after the above two steps, keep only the first 100000 most frequent tokens.

In [17]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

### Gensim doc2bow
For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4015, 1)]

In [20]:
dictionary.doc2bow(['call','wit'])

[(6, 1), (7, 1)]

In [21]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


### TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [22]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


### Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"warn" + 0.014*"driver" + 0.014*"flood" + 0.012*"street" + 0.011*"road" + 0.010*"babi" + 0.009*"east"
Topic: 1 
Words: 0.024*"world" + 0.018*"test" + 0.016*"interview" + 0.014*"australia" + 0.012*"leagu" + 0.010*"close" + 0.009*"darwin" + 0.009*"game" + 0.009*"take" + 0.009*"deal"
Topic: 2 
Words: 0.018*"council" + 0.018*"countri" + 0.017*"nation" + 0.015*"tasmanian" + 0.015*"hour" + 0.014*"state" + 0.013*"say" + 0.011*"citi" + 0.011*"news" + 0.011*"plan"
Topic: 3 
Words: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest" + 0.012*"guilti" + 0.011*"search" + 0.011*"john" + 0.011*"water" + 0.010*"continu"
Topic: 4 
Words: 0.028*"elect" + 0.017*"live" + 0.017*"famili" + 0.017*"say" + 0.016*"chang" + 0.011*"park" + 0.011*"polit" + 0.010*"leader" + 0.010*"christma" + 0.010*"parti"
Topic: 5 
Words: 0.035*"trump" + 0.028*"queensland" + 0.025*"australia" + 0.015*"tasmania" + 0.014*"turnbul" + 0

### Running LDA using TF-IDF

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.015*"countri" + 0.013*"hour" + 0.008*"health" + 0.007*"fund" + 0.006*"school" + 0.005*"council" + 0.005*"plan" + 0.005*"servic" + 0.005*"budget" + 0.005*"say"
Topic: 1 Word: 0.008*"kill" + 0.007*"christma" + 0.007*"violenc" + 0.007*"michael" + 0.006*"domest" + 0.006*"syria" + 0.006*"islam" + 0.005*"andrew" + 0.005*"toni" + 0.005*"capit"
Topic: 2 Word: 0.013*"govern" + 0.009*"sport" + 0.007*"climat" + 0.007*"asylum" + 0.006*"chang" + 0.006*"seeker" + 0.005*"social" + 0.005*"explain" + 0.005*"respons" + 0.004*"breakfast"
Topic: 3 Word: 0.013*"market" + 0.010*"share" + 0.007*"friday" + 0.007*"octob" + 0.006*"tuesday" + 0.006*"australian" + 0.005*"april" + 0.005*"wrap" + 0.005*"open" + 0.005*"australia"
Topic: 4 Word: 0.013*"podcast" + 0.013*"drum" + 0.012*"turnbul" + 0.007*"septemb" + 0.007*"cricket" + 0.007*"malcolm" + 0.006*"jam" + 0.005*"interview" + 0.005*"appeal" + 0.005*"girl"
Topic: 5 Word: 0.017*"polic" + 0.012*"crash" + 0.012*"woman" + 0.011*"charg" + 0.010*"inte

### Performance evaluation by classifying sample document using LDA Bag of Words model
     We will check where our test document would be classified.

In [26]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))



Score: 0.3140479326248169	 
Topic: 0.032*"govern" + 0.019*"market" + 0.014*"australian" + 0.013*"share" + 0.013*"break" + 0.011*"busi" + 0.011*"high" + 0.011*"bank" + 0.010*"port" + 0.009*"research"

Score: 0.2796989679336548	 
Topic: 0.016*"health" + 0.015*"rural" + 0.012*"servic" + 0.012*"price" + 0.012*"hous" + 0.011*"power" + 0.011*"concern" + 0.011*"communiti" + 0.010*"report" + 0.009*"farm"

Score: 0.2662428915500641	 
Topic: 0.028*"elect" + 0.017*"live" + 0.017*"famili" + 0.017*"say" + 0.016*"chang" + 0.011*"park" + 0.011*"polit" + 0.010*"leader" + 0.010*"christma" + 0.010*"parti"

Score: 0.02000545524060726	 
Topic: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest" + 0.012*"guilti" + 0.011*"search" + 0.011*"john" + 0.011*"water" + 0.010*"continu"

Score: 0.020002368837594986	 
Topic: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"warn" + 0.014*"driver" + 0.014*"flood" + 0.012*"street" + 0.011*"road" + 0.010*"babi" + 0.009*"east"

Score: 0

>Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate           classification.

### Performance evaluation by classifying sample document using LDA TF-IDF model.

In [27]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5569300651550293	 
Topic: 0.007*"flood" + 0.007*"marriag" + 0.006*"street" + 0.006*"novemb" + 0.006*"queensland" + 0.005*"david" + 0.005*"cyclon" + 0.005*"outback" + 0.004*"rain" + 0.004*"price"

Score: 0.28305596113204956	 
Topic: 0.013*"podcast" + 0.013*"drum" + 0.012*"turnbul" + 0.007*"septemb" + 0.007*"cricket" + 0.007*"malcolm" + 0.006*"jam" + 0.005*"interview" + 0.005*"appeal" + 0.005*"girl"

Score: 0.02000384032726288	 
Topic: 0.015*"countri" + 0.013*"hour" + 0.008*"health" + 0.007*"fund" + 0.006*"school" + 0.005*"council" + 0.005*"plan" + 0.005*"servic" + 0.005*"budget" + 0.005*"say"

Score: 0.020002331584692	 
Topic: 0.017*"polic" + 0.012*"crash" + 0.012*"woman" + 0.011*"charg" + 0.010*"interview" + 0.009*"driver" + 0.008*"shoot" + 0.008*"death" + 0.008*"die" + 0.007*"fatal"

Score: 0.020001739263534546	 
Topic: 0.008*"kill" + 0.007*"christma" + 0.007*"violenc" + 0.007*"michael" + 0.006*"domest" + 0.006*"syria" + 0.006*"islam" + 0.005*"andrew" + 0.005*"toni" + 0.005*

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

## Testing model on unseen document

In [29]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999728202819824	 Topic: 0.028*"elect" + 0.017*"live" + 0.017*"famili" + 0.017*"say" + 0.016*"chang"
Score: 0.1833333522081375	 Topic: 0.032*"govern" + 0.019*"market" + 0.014*"australian" + 0.013*"share" + 0.013*"break"
Score: 0.18333321809768677	 Topic: 0.035*"trump" + 0.028*"queensland" + 0.025*"australia" + 0.015*"tasmania" + 0.014*"turnbul"
Score: 0.1833287924528122	 Topic: 0.024*"world" + 0.018*"test" + 0.016*"interview" + 0.014*"australia" + 0.012*"leagu"
Score: 0.01667126454412937	 Topic: 0.023*"death" + 0.020*"hospit" + 0.016*"trial" + 0.014*"warn" + 0.014*"driver"
Score: 0.016668619588017464	 Topic: 0.016*"health" + 0.015*"rural" + 0.012*"servic" + 0.012*"price" + 0.012*"hous"
Score: 0.016667502000927925	 Topic: 0.023*"perth" + 0.020*"women" + 0.018*"miss" + 0.018*"sydney" + 0.013*"protest"
Score: 0.01666666753590107	 Topic: 0.018*"council" + 0.018*"countri" + 0.017*"nation" + 0.015*"tasmanian" + 0.015*"hour"
Score: 0.01666666753590107	 Topic: 0.031*"charg" + 0.029*"